In [1]:
import pandas as pd

In [2]:
movie_metadata_path = './Data/movie.metadata.tsv'
plot_summaries_path = './Data/plot_summaries.txt'

In [3]:
movie_metadata = pd.read_csv(
    movie_metadata_path, sep='\t', header=None, 
    names=[
        'movie_wikipedia_id', 'movie_freebase_id', 'movie_name', 
        'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres'
    ]
)

plot_summaries = pd.read_csv(
    plot_summaries_path, sep='\t', header=None, names=['movie_wikipedia_id', 'plot_summary'])

In [4]:
cmu_merged = pd.merge(movie_metadata, plot_summaries, on='movie_wikipedia_id')

In [5]:
len(cmu_merged['movie_name'].unique())

39914

We have 40K unique films with plots. Let's try to join it with external datasets with reviews

# Review datasets

## IMDB 50K

https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

It doesn't have movie name, so we cannot merge :(

## Movie lens

https://www.kaggle.com/datasets/grouplens/movielens-20m-dataset?select=rating.csv

In [6]:
import re

In [7]:
movielens_movies = pd.read_csv('movie_lens/movie.csv')
movielens_ratings = pd.read_csv('movie_lens/rating.csv')

In [8]:
movielens_mean_ratings = movielens_ratings.groupby('movieId', as_index=False)['rating'].mean()

In [9]:
movielens_merged = pd.merge(movielens_mean_ratings, movielens_movies, on='movieId')
movielens_merged = movielens_merged.drop(columns=['movieId', 'genres'])

In [10]:
len(movielens_merged)

26744

In [11]:
pattern = re.compile(r' \(\d{4}\)')

def remove_year(title):
    return pattern.sub('', title)

movielens_merged['title_without_year'] = movielens_merged['title'].apply(remove_year)

In [12]:
len(pd.merge(movielens_merged, cmu_merged, left_on='title_without_year', right_on='movie_name'))

12361

We can join only 12K films out of 40K in CMU :(

## IMDB from website

https://developer.imdb.com/non-commercial-datasets/

In [13]:
imdb_title_basics = pd.read_csv('imdb/title.basics.tsv', sep='\t')
imdb_title_ratings = pd.read_csv('imdb/title.ratings.tsv', sep='\t')

/var/folders/_s/zcd4d6nj55j53lv8b01ng45w0000gn/T/ipykernel_50391/371511669.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_title_basics = pd.read_csv('imdb/title.basics.tsv', sep='\t')


In [14]:
imdb_title_basics.query('primaryTitle != originalTitle').sample(5, random_state=42)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
6805769,tt28308058,short,Strained Brakes,Pinnat Kireenä,0,2023,\N,24,"Action,Comedy,Short"
4391169,tt1638945,video,Shame,Zonde,0,2010,\N,61,"Comedy,Drama,Romance"
5876905,tt2309770,movie,Fascination Galapagos 3D,Faszination Galapagos,0,2012,\N,\N,Documentary
221010,tt0230794,movie,Locksmith and Chancellor,Slesar i kantsler,0,1924,\N,54,Drama
6206811,tt25786454,tvMiniSeries,You Want Me to Steal You,Anata wa watashi ni otosaretai,0,2023,2023,30,Drama


`primaryTitle` is the title translated to English, so let's use it

In [15]:
imdb_title_basics = imdb_title_basics[['tconst', 'primaryTitle']]

In [16]:
imdb_merged = pd.merge(imdb_title_ratings, imdb_title_basics, on='tconst')
imdb_merged = imdb_merged.drop(columns=['tconst']).drop_duplicates(subset=['primaryTitle'])

In [17]:
imdb_merged.head()

,averageRating,numVotes,primaryTitle
0,5.7,2007,Carmencita
1,5.8,269,Le clown et ses chiens
2,6.5,1912,Pauvre Pierrot
3,5.5,178,Un bon bock
4,6.2,2692,Blacksmith Scene


In [18]:
len(imdb_merged), len(imdb_merged['primaryTitle'].unique())

(1021996, 1021996)

In [19]:
cmu_with_reviews = pd.merge(imdb_merged, cmu_merged, left_on='primaryTitle', right_on='movie_name', how='right')
cmu_with_reviews = cmu_with_reviews.drop(columns=['primaryTitle'])

In [20]:
cmu_with_reviews['averageRating'].notna().mean()

0.8171263387356649

So we have average ratings for 81% of movies!

# Merge Movie lens and IMDB

In [21]:
cmu_and_movielens = pd.merge(
    movielens_merged, cmu_with_reviews, left_on='title_without_year', right_on='movie_name', how='right')
cmu_and_movielens = cmu_and_movielens.drop(columns=['title', 'title_without_year'])

In [22]:
import numpy as np

In [23]:
cmu_and_movielens['movie_rating'] = np.where(cmu_and_movielens['averageRating'].notna(), 
                                             cmu_and_movielens['averageRating'], 
                                             cmu_and_movielens['rating'])

In [24]:
cmu_and_movielens = cmu_and_movielens.drop(columns=['rating', 'averageRating'])

In [25]:
cmu_and_movielens['movie_rating'].notna().mean()

0.8280770815214904

After adding data from movielens, we increased not nan values by 1%

Now we have ratings data for 83% of movies

## IMDB Movie Ratings Dataset

https://www.kaggle.com/datasets/thedevastator/imdb-movie-ratings-dataset

In [26]:
imdb_movie_data = pd.read_csv('movie_data.csv')[['movie_title', 'num_voted_users', 'imdb_score']]

In [27]:
imdb_movie_data['movie_title'] = imdb_movie_data['movie_title'].apply(lambda x: x.strip())

In [28]:
cmu_and_new_imdb = pd.merge(
    imdb_movie_data, cmu_and_movielens, left_on='movie_title', right_on='movie_name', how='right')
cmu_and_new_imdb = cmu_and_new_imdb.drop(columns=['movie_title'])

In [29]:
cmu_and_new_imdb['movie_rating'] = np.where(cmu_and_new_imdb['movie_rating'].notna(), 
                                             cmu_and_new_imdb['movie_rating'], 
                                             cmu_and_new_imdb['imdb_score'])

In [30]:
cmu_and_new_imdb['movie_rating'].notna().mean()

0.829665962713173

In [31]:
cmu_and_new_imdb['n_votes'] = np.where(cmu_and_new_imdb['numVotes'].notna(), 
                                             cmu_and_new_imdb['numVotes'], 
                                             cmu_and_new_imdb['num_voted_users'])

In [32]:
cmu_and_new_imdb['n_votes'].notna().mean()

0.8253059926879669

So, after merging data from three review datasets, we have average score for 83% of movies

We used Movie Lens dataset and two IMDB datasets